######

In [1]:
%run ../helpers/00-llm.ipynb

In [2]:
from helpers.llm import initialize_llm, logger, pretty_print_docs

llm, _, embeddings = initialize_llm() 

INFO:helpers.llm:Using AzureOpenAI.


In [3]:
from langchain_experimental.utilities import PythonREPL
from langchain_community.tools import DuckDuckGoSearchRun,WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Executa Código Python

In [4]:
python_repl = PythonREPL()
result = python_repl.run("print(5 * 5)")
print(result)

Python REPL can execute arbitrary code. Use with caution.


25



# Busca na Web

In [5]:
ddg_search = DuckDuckGoSearchRun()

query = "Qual é a capital da Guiana?"
search_results = ddg_search.run(query)

/home/grassato/langchain-for-beginners/.venv/lib/python3.12/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


In [6]:
print("Resultados da busca:", search_results)

Resultados da busca: May 16, 2025 · Qual é a capital da Guiana e por que é assim? A Guiana, um país fascinante localizado na costa nordeste da América do Sul, encanta com sua diversidade cultural e natural. … Dec 19, 2024 · A capital da Guiana, Georgetown, é conhecida por sua arquitetura colonial e seus mercados vibrantes. A Guiana é um dos poucos países do mundo onde a água potável é … Sep 2, 2024 · A taxa de criminalidade na Guiana é elevada, incluindo roubos violentos e ataques à mão armada. Algumas zonas de Georgetown são particularmente inseguras, pelo que deverá … 5 days ago · 1. Georgetown (capital da Guiana) A capital da Guiana é a porta de entrada para os turistas que escolhem chegar ao país de avião, mas também é um importante ponto turístico … Apr 24, 2025 · 🏛️ Qual é a capital da Guiana Brasileira? A capital da Guiana Brasileira é Lisboa. Após a integração com o Brasil, a cidade passou a ser considerada a capital do novo estado brasileiro, …


# Consulta a Wikipedia

In [7]:
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
query = "Guiana"
wikipedia_results = wikipedia.run(query)

/home/grassato/langchain-for-beginners/.venv/lib/python3.12/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /home/grassato/langchain-for-beginners/.venv/lib/python3.12/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [8]:
print("Resultados da Wikipedia:", wikipedia_results)

Resultados da Wikipedia: Page: French Guiana
Summary: French Guiana, or Guyane in French, is an overseas department and region of France located on the northern coast of South America in the Guianas and the West Indies. Bordered by Suriname to the west and Brazil to the east and south, French Guiana covers a total area of 84,000 km2 (32,000 sq mi) and a land area of 83,534 km2 (32,253 sq mi). As of January 2025, it is home to 292,354 people.

French Guiana is the second-largest region in France, being approximately one-seventh the size of European France, and the largest outermost region within the European Union. It has a very low population density, with only 3.6 inhabitants per square kilometre (9.3/sq mi). About half of its residents live in its capital, Cayenne. Approximately 98.9% of French Guiana is covered by forests, much of it primeval rainforest. Guiana Amazonian Park, the largest national park in the European Union covers 41% of French Guiana's territory.
Since December 201

In [3]:
from langchain_tavily import TavilySearch

tavily = TavilySearch(
    max_results=5,
    topic="general"
)

query = "Quais documentos são necessários para vender um carro?"
tavily_results = tavily.run({"query": query})


In [4]:
for result in tavily_results['results']:
    print(f"Title: {result['title']}")
    print(f"Content: {result['content']}")
    print("_*_" * 150)

Title: Quais são os documentos necessários para vender o carro?
Content: Neste post, o Zul+ preparou um passo a passo completo para você para realizar a transação do seu veículo e te mostra quais são os documentos necessários para vender o carro. Quando a transação estiver concluída, é preciso juntar todos os documentos necessários para vender o carro, como o CRV, o CRLV, a identidade e um comprovante de endereço do comprador, laudo de vistoria em uma empresa credenciada, formulário do Renavam e comprovante de pagamento das taxas. No dia marcado, é necessário a apresentação do CRV antigo original com o reconhecimento de firma do vendedor e do comprador, além de todos os documentos enviados no processo de transferência.
_*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__*__

### Custom tools

In [12]:
from langchain.tools import tool
import requests

@tool
def prever_idade(nome: str) -> str:
    """Prevê a idade média com base em um nome."""
    try:
        resposta = requests.get(f"https://api.agify.io/?name={nome}")
        if resposta.status_code == 200:
            idade = resposta.json().get("age", "desconhecida")
            return f"A idade média para o nome {nome} é {idade} anos."
        else:
            return f"Erro ao consultar idade para {nome}."
    except Exception as e:
        return f"Erro de conexão: {str(e)}"

In [13]:
prever_idade.run("Vanessa")

'A idade média para o nome Vanessa é 42 anos.'

In [14]:
from langchain.tools import tool
from pydantic import BaseModel, Field

# Definindo o esquema de entrada para a ferramenta
class MultiplicacaoInput(BaseModel):
    x: int = Field(description="Primeiro número inteiro a ser multiplicado")
    y: int = Field(description="Segundo número inteiro a ser multiplicado")

@tool(args_schema=MultiplicacaoInput)
def multiplicar(x: int, y: int) -> int:
    """Multiplica dois números inteiros"""
    print(f"Multiplicando {x} por {y}")
    return x * y


In [15]:
multiplicar.run({"x": 9, "y": 10})

Multiplicando 9 por 10


90